<a href="https://colab.research.google.com/github/zchisholm/zenocode/blob/main/Codebase_RAG_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Success

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [10]:
def clone_repo(repo_url):
    repo_name = repo_url.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_name)

In [12]:
repo_link = 'https://github.com/zchisholm/ZensiReviews'

In [13]:
path = clone_repo(repo_link)

In [ ]:
# path = "/content/"+repo_url.split("/")[-1]

In [14]:
SUPPORTED_EXTENSIONS = [".py", ".js", ".tsx", ".ts", ".java", ".cpp"]

IGNORED_DIRS = ["node_modules", ".git", "dist", "__pycache__", ".next", ".vscode", "env", "venv"]

In [15]:
def get_file_content(file_path, repo_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None



In [16]:
def get_main_files_content(repo_path: str):
   """
   Get content of supported code files from the local repository.


   Args:
       repo_path: Path to the local repository


   Returns:
       List of dictionaries containing file names and contents
   """
   files_content = []


   try:
       for root, _, files in os.walk(repo_path):
           # Skip if current directory is in ignored directories
           if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
               continue


           # Process each file in current directory
           for file in files:
               file_path = os.path.join(root, file)
               if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                   file_content = get_file_content(file_path, repo_path)
                   if file_content:
                       files_content.append(file_content)


   except Exception as e:
       print(f"Error reading repository: {str(e)}")


   return files_content

In [25]:
file_content = get_main_files_content(path)

# Embeddings

In [17]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [18]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
embeddings

array([ 1.81737654e-02, -3.02657508e-03, -4.77465875e-02,  1.86379403e-02,
        3.14537995e-02,  1.87255293e-02, -1.52534274e-02, -6.77293688e-02,
       -1.26903653e-02,  1.28427576e-02,  5.80701306e-02,  4.00234833e-02,
        3.27073298e-02,  7.12998435e-02,  5.56373373e-02,  1.68628506e-02,
        6.97603747e-02, -5.02619930e-02,  6.13140827e-03, -1.46559235e-02,
       -4.51957993e-03,  4.82934639e-02, -2.53051296e-02, -1.97862904e-03,
       -4.36902530e-02, -2.41507161e-02,  1.29505759e-02, -3.78611824e-03,
       -2.05718316e-02,  1.09819308e-01,  3.07672890e-03, -2.80443169e-02,
       -1.55807249e-02, -1.24789868e-02,  1.75239131e-06, -2.93756695e-03,
       -1.43048428e-02,  4.88386713e-02, -6.21114224e-02,  2.95061413e-02,
       -1.40470508e-02,  2.20708270e-02,  1.13067888e-02,  4.70893271e-02,
        7.58305984e-03, -8.30314530e-05,  6.67821169e-02, -1.21320095e-02,
        4.39386303e-03,  2.47453637e-02,  1.02529004e-02, -6.54432410e-03,
       -5.53147821e-03, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [22]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pc_indexname = "zeno-code"
pinecone_index = pc.Index(pc_indexname)

In [23]:
vectorstore = PineconeVectorStore(index_name=pc_indexname, embedding=HuggingFaceEmbeddings())

<ipython-input-23-8740dea1d54e>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name=pc_indexname, embedding=HuggingFaceEmbeddings())
<ipython-input-23-8740dea1d54e>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name=pc_indexname, embedding=HuggingFaceEmbeddings())


In [26]:
# Insert the codebase embeddings into Pinecone

documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )
    documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name=pc_indexname,
    namespace=repo_link
)

<ipython-input-26-0439f7f9049a>:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [28]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [27]:
query = "How is the JavaScript parser used?"

In [29]:
query_embedding = get_huggingface_embeddings(query)

In [ ]:
top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5,
    include_metadata=True, namespace=repo_link)


In [ ]:
system_prompt = f""""""


llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content